# This Example shows the Prediction of Bike Flow in the NYC City using the deep learning model DeepSTN.

Find the details of the ST-ResNet model in the <a href="https://dl.acm.org/doi/10.5555/3298239.3298479">corresponding paper</a>

Details of the dataset can be found <a href="https://github.com/FIBLAB/DeepSTN">here</a>.

### Import Modules

In [23]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torchvision.transforms as transforms

from geotorchai.models.grid import STResNet, DeepSTN
from geotorchai.preprocessing.torch_df import SpatiotemporalDfToTorchData
from geotorchai.preprocessing import SparkRegistration, load_parquet_data
from geotorchai.preprocessing.raster import STManager as stm
from geotorchai.utility import TorchAdapter

# Import Apache Sedona
from sedona.spark import *

## Import PySpark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf, expr, array, concat


## Import distributed modules
from torch.utils.data import DistributedSampler, DataLoader
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from pyspark.ml.torch.distributor import TorchDistributor
from petastorm import TransformSpec

## Define spark session and Register with GeoTorchAI

In [ ]:
config = SedonaContext.builder().master(MASTER_URL).config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.4_2.12:1.4.1,'
           'org.datasyslab:geotools-wrapper:1.4.0-28.2').getOrCreate()

spark = SedonaContext.create(config)
sc = spark.sparkContext

### Register SparkSession with GeoTorchAI

In [24]:
SparkRegistration.set_spark_session(spark)

23/06/22 10:18:27 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.geom.Geometry, which is already registered.
23/06/22 10:18:27 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.index.SpatialIndex, which is already registered.
23/06/22 10:18:27 WARN SimpleFunctionRegistry: The function st_union_aggr replaced a previously registered function.
23/06/22 10:18:27 WARN SimpleFunctionRegistry: The function st_envelope_aggr replaced a previously registered function.
23/06/22 10:18:27 WARN SimpleFunctionRegistry: The function st_intersection_aggr replaced a previously registered function.


## Load Taxi Pick Up Data to Sedona

In [25]:
taxi_df = load_parquet_data("data/yellow_trip_10_fraction.parquet")
taxi_df.show(5)

+---------+-------------------+-------------------+---------------+-------------+------------------+---------------+---------+------------------+------------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|pickup_latitude|rate_code|store_and_fwd_flag| dropoff_longitude|dropoff_latitude|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|
+---------+-------------------+-------------------+---------------+-------------+------------------+---------------+---------+------------------+------------------+----------------+------------+-----------+---------+-------+----------+------------+------------+
|      CMT|2010-10-01 11:11:14|2010-10-01 11:25:51|              2|          3.0|        -73.962241|      40.779117|        1|                 N|-73.99491999999998|       40.755117|         CSH|       10.9|      0.

In [26]:
taxi_df = taxi_df.select("pickup_datetime", "pickup_longitude", "pickup_latitude")
taxi_df = taxi_df.filter(taxi_df.pickup_latitude >= 40.491370).filter(taxi_df.pickup_latitude <= 40.91553).filter(taxi_df.pickup_longitude >= -74.259090).filter(taxi_df.pickup_longitude <= -73.700180)
taxi_df.show(5)

+-------------------+------------------+---------------+
|    pickup_datetime|  pickup_longitude|pickup_latitude|
+-------------------+------------------+---------------+
|2010-10-01 11:11:14|        -73.962241|      40.779117|
|2010-10-01 11:11:56|        -73.988973|      40.757885|
|2010-10-01 14:41:37|-73.74765499999998|      40.772559|
|2010-10-01 09:37:06|        -73.959452|      40.763384|
|2010-10-01 22:26:46|        -73.987311|      40.719792|
+-------------------+------------------+---------------+
only showing top 5 rows



In [ ]:
layer = stm.getHexagonalLayer(taxi_df, col_lat='pickup_latitude', col_lon='pickup_longitude', fraction=0.5)
layer.to_html('hexagon-example.html')

## Convert into a Spatial DataFrame

In [8]:
taxi_df = stm.add_spatial_points(taxi_df, lat_column="pickup_latitude", lon_column="pickup_longitude", new_column_alias="geometry").drop(*("pickup_latitude", "pickup_longitude"))
taxi_df.show(5, False)

+-------------------+----------------------------+
|pickup_datetime    |geometry                    |
+-------------------+----------------------------+
|2010-10-01 02:46:21|POINT (-74.004524 40.722785)|
|2010-10-01 01:13:21|POINT (-74.001624 40.750705)|
|2010-10-01 13:53:20|POINT (-73.997414 40.722494)|
|2010-10-01 14:24:41|POINT (-73.996218 40.753552)|
|2010-10-01 22:19:39|POINT (-73.997439 40.741765)|
+-------------------+----------------------------+
only showing top 5 rows



## Convert into a Spatiotemporal Grid DataFrame

In [9]:
polygons = stm.get_grid_cell_polygons(taxi_df, "geometry", partitions_x=12, partitions_y=12)
taxi_df = stm.get_st_grid_dataframe(geo_df = taxi_df, geometry = "geometry", partitions_x = 12, partitions_y = 12, col_date = "pickup_datetime", step_duration_sec = 3600)
taxi_df.show(5)

+-------+------------+------------------+
|cell_id|_id_timestep|aggregated_feature|
+-------+------------+------------------+
|     65|         172|                 6|
|    102|         335|               164|
|     77|         516|               432|
|     92|         402|                66|
|     91|         344|                18|
+-------+------------+------------------+
only showing top 5 rows



In [10]:
layer = stm.getStGridLayer(taxi_df, timestamp_index=20, col_timestamp="_id_timestep", col_geometry="cell_geometry", col_feature="aggregated_feature", col_id="cell_id", polygons=polygons)
layer.to_html()

## Define parameters

In [11]:
learning_rate = 0.0002
batch_size = 32
epoch_nums = 20

### Convert Sedona DatFrame into GeoTorchAI Dataset

In [ ]:
num_timsteps = stm.get_temporal_steps_count(taxi_df, temporal_steps_column="_id_timestep")
full_dataset = SpatiotemporalDfToTorchData(taxi_df, "_id_timestep", "cell_id", ["aggregated_feature"], num_timsteps, map_height, map_width)
min_max_difference, min_max_sum = full_dataset.get_min_max_info()

### Method to Return Model

In [15]:
def get_model():
    map_height, map_width, nb_flow = 12, 12, 1   
    is_plus=False     
    is_pt=False
    drop=0.1

    ## Define Model
    model = DeepSTN(H=map_height, W=map_width, channel=1, is_plus=is_plus, is_pt=is_pt, dropVal=drop)
    return model

### Train the Model
Error will be high since the training is performed only for 20 epochs

In [ ]:
def train_one_epoch(model, train_loader, optimizer, loss_fn, device):
    model.train()
    for i, sample in enumerate(train_loader):
        X_c = sample["x_closeness"].type(torch.FloatTensor).to(device)
        X_p = sample["x_period"].type(torch.FloatTensor).to(device)
        X_t = sample["x_trend"].type(torch.FloatTensor).to(device)
        Y_batch = sample["y_data"].type(torch.FloatTensor).to(device)

        with torch.set_grad_enabled(True):
            optimizer.zero_grad()

            # Forward pass
            outputs = model(X_c, X_p, X_t)
            loss = loss_fn(outputs, Y_batch)

            # Backward and optimize
            loss.backward()
            optimizer.step()
    return loss.item()

In [ ]:
def train_model(model, loader, device):
    ## Define hyper-parameters
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for e in range(epoch_nums):
        epoch_loss = train_one_epoch(model, loader, optimizer, loss_fn, device)
        print('Epoch [{}/{}], Training Loss: {:.4f}'.format(e + 1, epoch_nums, epoch_loss))

In [ ]:
def train_distributed(use_gpu):
    backend = "nccl" if use_gpu else "gloo"
    dist.init_process_group(backend)
    device = int(os.environ["LOCAL_RANK"]) if use_gpu  else "cpu"
    model = get_model().to(device)
    model_ddp = DDP(model)
    sampler = DistributedSampler(full_dataset)
    loader = DataLoader(full_dataset, batch_size=batch_size, sampler=sampler)

    train_model(model_ddp, loader, device)

## Start Distributed Training

In [17]:
distributor = TorchDistributor(num_processes=2, local_mode=True, use_gpu=False)
distributor.run(train_distributed, False)

Epoch [1/20], Training Loss: 0.9906
Mean validation loss: 0.4560351073741913
Epoch [2/20], Training Loss: 0.3084
Mean validation loss: 0.1550356149673462
Epoch [3/20], Training Loss: 0.1318
Mean validation loss: 0.05485467612743378
Epoch [4/20], Training Loss: 0.0520
Mean validation loss: 0.01706523820757866
Epoch [5/20], Training Loss: 0.0179
Mean validation loss: 0.011119382455945015
Epoch [6/20], Training Loss: 0.0111
Mean validation loss: 0.012275331653654575
Epoch [7/20], Training Loss: 0.0093
Mean validation loss: 0.013477280735969543
Epoch [8/20], Training Loss: 0.0099
Mean validation loss: 0.013310907408595085
Epoch [9/20], Training Loss: 0.0083
Mean validation loss: 0.01162323635071516
Epoch [10/20], Training Loss: 0.0079
Mean validation loss: 0.009120878763496876
Epoch [11/20], Training Loss: 0.0061
Mean validation loss: 0.00666249543428421
Epoch [12/20], Training Loss: 0.0048
Mean validation loss: 0.00509181572124362
Epoch [13/20], Training Loss: 0.0043
Mean validation loss: